# Task 6

In [1]:
import numpy as np
import cv2 as cv
from __future__ import print_function
import argparse


In [24]:
cap = cv.VideoCapture('11.mp4')
fgbg = cv.createBackgroundSubtractorKNN()

while(1):
    ret, frame = cap.read()
    if frame is None:
        break

    fgmask = fgbg.apply(frame)

    # get the frame number and write it on the current frame
    cv.rectangle(frame, (10, 2), (100, 20), (255, 255, 255), -1)
    cv.putText(frame, str(cap.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

    # Loại bỏ nhiễu
    kernel = np.ones((5, 5), np.uint8) 
    fgmask = cv.erode(fgmask, kernel, iterations=1)
    fgmask = cv.dilate(fgmask, kernel, iterations=12)
    fgmask = cv.GaussianBlur(fgmask, (3, 3), 0)
    fgmask = cv.morphologyEx(fgmask, cv.MORPH_CLOSE, kernel)
    # _ là ret (ignoring value)
    _, fgmask = cv.threshold(fgmask, 130, 255, cv.THRESH_BINARY)

    fgmask = cv.Canny(fgmask, 20, 200)
    contours, _ = cv.findContours(
        fgmask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)  # _ là hierarchy
    for i in range(len(contours)):
        (x, y, w, h) = cv.boundingRect(contours[i])
        if cv.contourArea(contours[i]) > 2000:
            cv.drawContours(fgmask, contours[i], 0, (0, 0, 255), 8)
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 4)
    
    cv.imshow('fgmask', fgmask)
    cv.imshow('frame', frame)

    k = cv.waitKey(25) & 0xff
    if k == 27 or k == 'q':
        break


cap.release()
cv.destroyAllWindows()
